In [1]:
%matplotlib inline

import os
import sys
genre_class_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print("Loading Genre_Classification system from path", genre_class_path)
sys.path.append(genre_class_path)

import numpy as np
import pandas as pd
import tensorflow as tf
import json

Loading AI Lit system from path /mnt/d/UNIV NOTES/Y4S1/DSA4199/ai-lit


2022-10-25 15:53:02.972956: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-25 15:53:02.973088: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/kathyij74/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
dataset_wkspc = os.path.join(genre_class_path, 'workspace', 'gb_input')
data_path = os.path.join(dataset_wkspc, 'data_index.json')

with open(data_path) as f:
    data_json = json.load(f)

In [4]:
data_json[0]['body']

'project gutenberg\'s the house on the borderland, by william hope hodgson\r\n\r\nthis ebook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  you may copy it, give it away or\r\nre-use it under the terms of the project gutenberg license included\r\nwith this ebook or online at www.gutenberg.org\r\n\r\n\r\ntitle: the house on the borderland\r\n\r\nauthor: william hope hodgson\r\n\r\nrelease date: november 10, 2003 [ebook #10002]\r\nlast updated: january 19, 2009\r\n\r\nlanguage: english\r\n\r\ncharacter set encoding: ascii\r\n\r\n*** start of this project gutenberg ebook the house on the borderland ***\r\n\r\n\r\n\r\n\r\nproduced by suzanne shell, sjaani and pg distributed proofreaders\r\n\r\n\r\n\r\n\r\nthe house on the borderland\r\n\r\nwilliam hope hodgson\r\n\r\n\r\n_from the manuscript discovered in 1877 by messrs. tonnison and\r\nberreggnog in the ruins that lie to the south of the village of\r\nkraighten, in the west of ireland. set out

In [5]:
data_json[0]

{'index': '10002',
 'indexfile': 'Gutenberg/cache/epub/10002/pg10002.rdf',
 'title': 'the house on the borderland',
 'author': 'hodgson, william hope',
 'subjects': ['Science fiction'],
 'bodyfile': 'Gutenberg/aleph.gutenberg.org/1/0/0/0/10002/10002.zip',
 'body': 'project gutenberg\'s the house on the borderland, by william hope hodgson\r\n\r\nthis ebook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  you may copy it, give it away or\r\nre-use it under the terms of the project gutenberg license included\r\nwith this ebook or online at www.gutenberg.org\r\n\r\n\r\ntitle: the house on the borderland\r\n\r\nauthor: william hope hodgson\r\n\r\nrelease date: november 10, 2003 [ebook #10002]\r\nlast updated: january 19, 2009\r\n\r\nlanguage: english\r\n\r\ncharacter set encoding: ascii\r\n\r\n*** start of this project gutenberg ebook the house on the borderland ***\r\n\r\n\r\n\r\n\r\nproduced by suzanne shell, sjaani and pg distributed proofreade

In [6]:
data_try = data_json[0]['body']

# Step - a : Remove blank rows if any.
# data_try.dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
data_try = data_try.lower()
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
data_try= word_tokenize(data_try)
# Step - d : POS Tagging
data_try = pos_tag(data_try)

In [7]:
print(data_try)

[('project', 'NN'), ('gutenberg', 'NN'), ("'s", 'POS'), ('the', 'DT'), ('house', 'NN'), ('on', 'IN'), ('the', 'DT'), ('borderland', 'NN'), (',', ','), ('by', 'IN'), ('william', 'JJ'), ('hope', 'NN'), ('hodgson', 'NN'), ('this', 'DT'), ('ebook', 'NN'), ('is', 'VBZ'), ('for', 'IN'), ('the', 'DT'), ('use', 'NN'), ('of', 'IN'), ('anyone', 'NN'), ('anywhere', 'RB'), ('at', 'IN'), ('no', 'DT'), ('cost', 'NN'), ('and', 'CC'), ('with', 'IN'), ('almost', 'RB'), ('no', 'DT'), ('restrictions', 'NNS'), ('whatsoever', 'RB'), ('.', '.'), ('you', 'PRP'), ('may', 'MD'), ('copy', 'VB'), ('it', 'PRP'), (',', ','), ('give', 'VB'), ('it', 'PRP'), ('away', 'RB'), ('or', 'CC'), ('re-use', 'VB'), ('it', 'PRP'), ('under', 'IN'), ('the', 'DT'), ('terms', 'NNS'), ('of', 'IN'), ('the', 'DT'), ('project', 'NN'), ('gutenberg', 'NN'), ('license', 'NN'), ('included', 'VBD'), ('with', 'IN'), ('this', 'DT'), ('ebook', 'NN'), ('or', 'CC'), ('online', 'NN'), ('at', 'IN'), ('www.gutenberg.org', 'JJ'), ('title', 'NN'), ('

In [8]:
data_tags = [t[1] for t in data_try]

In [9]:
print(data_tags)

['NN', 'NN', 'POS', 'DT', 'NN', 'IN', 'DT', 'NN', ',', 'IN', 'JJ', 'NN', 'NN', 'DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'NN', 'RB', 'IN', 'DT', 'NN', 'CC', 'IN', 'RB', 'DT', 'NNS', 'RB', '.', 'PRP', 'MD', 'VB', 'PRP', ',', 'VB', 'PRP', 'RB', 'CC', 'VB', 'PRP', 'IN', 'DT', 'NNS', 'IN', 'DT', 'NN', 'NN', 'NN', 'VBD', 'IN', 'DT', 'NN', 'CC', 'NN', 'IN', 'JJ', 'NN', ':', 'DT', 'NN', 'IN', 'DT', 'NN', 'NN', ':', 'NN', 'VBP', 'NN', 'NN', 'NN', ':', 'JJ', 'CD', ',', 'CD', 'NNP', 'VBD', '#', 'CD', 'NN', 'JJ', 'JJ', ':', 'JJ', 'CD', ',', 'CD', 'NN', ':', 'JJ', 'NN', 'VBN', 'NN', ':', 'NN', 'NNP', 'NNP', 'NNP', 'NN', 'IN', 'DT', 'NN', 'VBZ', 'VB', 'DT', 'NN', 'IN', 'DT', 'NN', 'NNP', 'NNP', 'NNP', 'VBN', 'IN', 'NN', 'NN', ',', 'NN', 'CC', 'NN', 'VBD', 'NNS', 'DT', 'NN', 'IN', 'DT', 'NN', 'VBZ', 'VBP', 'NN', 'IN', 'DT', 'NN', 'VBN', 'IN', 'CD', 'IN', 'NNS', 'NN', 'CC', 'NN', 'IN', 'DT', 'NNS', 'WDT', 'VBP', 'TO', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNS', ',', 'IN', 'DT', 'NN', 'IN', 'NN', '.',

In [6]:
from nltk import help
from nltk.util import ngrams

In [7]:
download('tagsets')
help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

[nltk_data] Downloading package tagsets to
[nltk_data]     /home/kathyij74/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [8]:
taglist = ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT',
          'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP',
          'WP$', 'WRB']

In [13]:
two_gram_try = list(ngrams(data_tags, 2))

In [14]:
two_gram_try

[('NN', 'NN'),
 ('NN', 'POS'),
 ('POS', 'DT'),
 ('DT', 'NN'),
 ('NN', 'IN'),
 ('IN', 'DT'),
 ('DT', 'NN'),
 ('NN', ','),
 (',', 'IN'),
 ('IN', 'JJ'),
 ('JJ', 'NN'),
 ('NN', 'NN'),
 ('NN', 'DT'),
 ('DT', 'NN'),
 ('NN', 'VBZ'),
 ('VBZ', 'IN'),
 ('IN', 'DT'),
 ('DT', 'NN'),
 ('NN', 'IN'),
 ('IN', 'NN'),
 ('NN', 'RB'),
 ('RB', 'IN'),
 ('IN', 'DT'),
 ('DT', 'NN'),
 ('NN', 'CC'),
 ('CC', 'IN'),
 ('IN', 'RB'),
 ('RB', 'DT'),
 ('DT', 'NNS'),
 ('NNS', 'RB'),
 ('RB', '.'),
 ('.', 'PRP'),
 ('PRP', 'MD'),
 ('MD', 'VB'),
 ('VB', 'PRP'),
 ('PRP', ','),
 (',', 'VB'),
 ('VB', 'PRP'),
 ('PRP', 'RB'),
 ('RB', 'CC'),
 ('CC', 'VB'),
 ('VB', 'PRP'),
 ('PRP', 'IN'),
 ('IN', 'DT'),
 ('DT', 'NNS'),
 ('NNS', 'IN'),
 ('IN', 'DT'),
 ('DT', 'NN'),
 ('NN', 'NN'),
 ('NN', 'NN'),
 ('NN', 'VBD'),
 ('VBD', 'IN'),
 ('IN', 'DT'),
 ('DT', 'NN'),
 ('NN', 'CC'),
 ('CC', 'NN'),
 ('NN', 'IN'),
 ('IN', 'JJ'),
 ('JJ', 'NN'),
 ('NN', ':'),
 (':', 'DT'),
 ('DT', 'NN'),
 ('NN', 'IN'),
 ('IN', 'DT'),
 ('DT', 'NN'),
 ('NN', 'NN'),


In [15]:
[t for t in two_gram_try if ((t[0] in taglist) and (t[1] in taglist))]

[('NN', 'NN'),
 ('NN', 'POS'),
 ('POS', 'DT'),
 ('DT', 'NN'),
 ('NN', 'IN'),
 ('IN', 'DT'),
 ('DT', 'NN'),
 ('IN', 'JJ'),
 ('JJ', 'NN'),
 ('NN', 'NN'),
 ('NN', 'DT'),
 ('DT', 'NN'),
 ('NN', 'VBZ'),
 ('VBZ', 'IN'),
 ('IN', 'DT'),
 ('DT', 'NN'),
 ('NN', 'IN'),
 ('IN', 'NN'),
 ('NN', 'RB'),
 ('RB', 'IN'),
 ('IN', 'DT'),
 ('DT', 'NN'),
 ('NN', 'CC'),
 ('CC', 'IN'),
 ('IN', 'RB'),
 ('RB', 'DT'),
 ('DT', 'NNS'),
 ('NNS', 'RB'),
 ('PRP', 'MD'),
 ('MD', 'VB'),
 ('VB', 'PRP'),
 ('VB', 'PRP'),
 ('PRP', 'RB'),
 ('RB', 'CC'),
 ('CC', 'VB'),
 ('VB', 'PRP'),
 ('PRP', 'IN'),
 ('IN', 'DT'),
 ('DT', 'NNS'),
 ('NNS', 'IN'),
 ('IN', 'DT'),
 ('DT', 'NN'),
 ('NN', 'NN'),
 ('NN', 'NN'),
 ('NN', 'VBD'),
 ('VBD', 'IN'),
 ('IN', 'DT'),
 ('DT', 'NN'),
 ('NN', 'CC'),
 ('CC', 'NN'),
 ('NN', 'IN'),
 ('IN', 'JJ'),
 ('JJ', 'NN'),
 ('DT', 'NN'),
 ('NN', 'IN'),
 ('IN', 'DT'),
 ('DT', 'NN'),
 ('NN', 'NN'),
 ('NN', 'VBP'),
 ('VBP', 'NN'),
 ('NN', 'NN'),
 ('NN', 'NN'),
 ('JJ', 'CD'),
 ('CD', 'NNP'),
 ('NNP', 'VBD'),
 ('C

In [16]:
three_gram_try = list(ngrams(data_tags, 3))

In [17]:
three_gram_try

[('NN', 'NN', 'POS'),
 ('NN', 'POS', 'DT'),
 ('POS', 'DT', 'NN'),
 ('DT', 'NN', 'IN'),
 ('NN', 'IN', 'DT'),
 ('IN', 'DT', 'NN'),
 ('DT', 'NN', ','),
 ('NN', ',', 'IN'),
 (',', 'IN', 'JJ'),
 ('IN', 'JJ', 'NN'),
 ('JJ', 'NN', 'NN'),
 ('NN', 'NN', 'DT'),
 ('NN', 'DT', 'NN'),
 ('DT', 'NN', 'VBZ'),
 ('NN', 'VBZ', 'IN'),
 ('VBZ', 'IN', 'DT'),
 ('IN', 'DT', 'NN'),
 ('DT', 'NN', 'IN'),
 ('NN', 'IN', 'NN'),
 ('IN', 'NN', 'RB'),
 ('NN', 'RB', 'IN'),
 ('RB', 'IN', 'DT'),
 ('IN', 'DT', 'NN'),
 ('DT', 'NN', 'CC'),
 ('NN', 'CC', 'IN'),
 ('CC', 'IN', 'RB'),
 ('IN', 'RB', 'DT'),
 ('RB', 'DT', 'NNS'),
 ('DT', 'NNS', 'RB'),
 ('NNS', 'RB', '.'),
 ('RB', '.', 'PRP'),
 ('.', 'PRP', 'MD'),
 ('PRP', 'MD', 'VB'),
 ('MD', 'VB', 'PRP'),
 ('VB', 'PRP', ','),
 ('PRP', ',', 'VB'),
 (',', 'VB', 'PRP'),
 ('VB', 'PRP', 'RB'),
 ('PRP', 'RB', 'CC'),
 ('RB', 'CC', 'VB'),
 ('CC', 'VB', 'PRP'),
 ('VB', 'PRP', 'IN'),
 ('PRP', 'IN', 'DT'),
 ('IN', 'DT', 'NNS'),
 ('DT', 'NNS', 'IN'),
 ('NNS', 'IN', 'DT'),
 ('IN', 'DT', 'NN')

In [9]:
# create a function to filter the n-gram
taglist = ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT',
          'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP',
          'WP$', 'WRB']
def gram_select(text, n):
    text = text.lower()
    text = word_tokenize(text)
    text = pos_tag(text)
    data_tags = [t[1] for t in text]
    tags_list = list(ngrams(data_tags, n))
    tags_list = [t for t in tags_list if ((t[0] in taglist) and (t[1] in taglist))]
    return tags_list

In [20]:
from collections import Counter
bigram_list = []
for entry in data_json:
    bigram_list.append({'index': entry['index'], 'title': entry['title'], 
                        'author': entry['author'], 'subjects': entry['subjects'], 
                        'body': Counter(gram_select(entry['body'], 2))})

In [35]:
bigram_list_updated = bigram_list.copy()

In [36]:
for entry in bigram_list_updated:
    new_dict = {}
    for key in entry['body'].keys():
        new_dict[str(key)] = entry['body'][key]
    entry['body'] = new_dict

In [37]:
len(bigram_list_updated)

3889

In [42]:
bigram_list_updated[16]

{'index': '10434',
 'title': 'wyandottã©; or, the hutted knoll: a tale',
 'author': 'cooper, james fenimore',
 'subjects': ['Historical fiction',
  'War stories',
  'New York (State) -- History -- Revolution, 1775-1783 -- Fiction',
  'Tuscarora Indians -- Fiction',
  'Otsego County (N.Y.) -- Fiction'],
 'body': {"('DT', 'NN')": 11690,
  "('NN', 'NN')": 1470,
  "('IN', 'NNS')": 678,
  "('NNS', 'RB')": 213,
  "('RB', 'VBP')": 143,
  "('VBP', 'DT')": 347,
  "('NN', 'VBZ')": 618,
  "('VBZ', 'IN')": 178,
  "('IN', 'DT')": 8902,
  "('NN', 'IN')": 6197,
  "('IN', 'NN')": 3352,
  "('NN', 'RB')": 592,
  "('RB', 'IN')": 1063,
  "('NN', 'CC')": 902,
  "('CC', 'IN')": 371,
  "('IN', 'RB')": 366,
  "('RB', 'DT')": 450,
  "('DT', 'NNS')": 2218,
  "('PRP', 'MD')": 860,
  "('MD', 'VB')": 1837,
  "('VB', 'PRP')": 894,
  "('PRP', 'RB')": 423,
  "('RB', 'CC')": 76,
  "('CC', 'VB')": 341,
  "('PRP', 'IN')": 462,
  "('NNS', 'IN')": 1816,
  "('NN', 'VBD')": 2450,
  "('VBD', 'IN')": 949,
  "('CC', 'NN')": 83

In [21]:
len(bigram_list)

3889

In [38]:
bigram_list_updated[0]

{'index': '10002',
 'title': 'the house on the borderland',
 'author': 'hodgson, william hope',
 'subjects': ['Science fiction'],
 'body': {"('NN', 'NN')": 497,
  "('NN', 'POS')": 52,
  "('POS', 'DT')": 1,
  "('DT', 'NN')": 3649,
  "('NN', 'IN')": 2026,
  "('IN', 'DT')": 3376,
  "('IN', 'JJ')": 418,
  "('JJ', 'NN')": 1965,
  "('NN', 'DT')": 42,
  "('NN', 'VBZ')": 155,
  "('VBZ', 'IN')": 50,
  "('IN', 'NN')": 1190,
  "('NN', 'RB')": 143,
  "('RB', 'IN')": 457,
  "('NN', 'CC')": 254,
  "('CC', 'IN')": 100,
  "('IN', 'RB')": 154,
  "('RB', 'DT')": 159,
  "('DT', 'NNS')": 565,
  "('NNS', 'RB')": 51,
  "('PRP', 'MD')": 128,
  "('MD', 'VB')": 403,
  "('VB', 'PRP')": 140,
  "('PRP', 'RB')": 63,
  "('RB', 'CC')": 46,
  "('CC', 'VB')": 138,
  "('PRP', 'IN')": 156,
  "('NNS', 'IN')": 384,
  "('NN', 'VBD')": 1864,
  "('VBD', 'IN')": 533,
  "('CC', 'NN')": 317,
  "('NN', 'VBP')": 115,
  "('VBP', 'NN')": 27,
  "('JJ', 'CD')": 19,
  "('CD', 'NNP')": 4,
  "('NNP', 'VBD')": 3,
  "('CD', 'NN')": 76,
  

In [13]:
list(bigram_list[0]['body'])

[('NN', 'NN'),
 ('NN', 'POS'),
 ('POS', 'DT'),
 ('DT', 'NN'),
 ('NN', 'IN'),
 ('IN', 'DT'),
 ('IN', 'JJ'),
 ('JJ', 'NN'),
 ('NN', 'DT'),
 ('NN', 'VBZ'),
 ('VBZ', 'IN'),
 ('IN', 'NN'),
 ('NN', 'RB'),
 ('RB', 'IN'),
 ('NN', 'CC'),
 ('CC', 'IN'),
 ('IN', 'RB'),
 ('RB', 'DT'),
 ('DT', 'NNS'),
 ('NNS', 'RB'),
 ('PRP', 'MD'),
 ('MD', 'VB'),
 ('VB', 'PRP'),
 ('PRP', 'RB'),
 ('RB', 'CC'),
 ('CC', 'VB'),
 ('PRP', 'IN'),
 ('NNS', 'IN'),
 ('NN', 'VBD'),
 ('VBD', 'IN'),
 ('CC', 'NN'),
 ('NN', 'VBP'),
 ('VBP', 'NN'),
 ('JJ', 'CD'),
 ('CD', 'NNP'),
 ('NNP', 'VBD'),
 ('CD', 'NN'),
 ('NN', 'JJ'),
 ('JJ', 'JJ'),
 ('NN', 'VBN'),
 ('VBN', 'NN'),
 ('NN', 'NNP'),
 ('NNP', 'NNP'),
 ('NNP', 'NN'),
 ('VBZ', 'VB'),
 ('VB', 'DT'),
 ('NNP', 'VBN'),
 ('VBN', 'IN'),
 ('VBD', 'NNS'),
 ('NNS', 'DT'),
 ('VBZ', 'VBP'),
 ('IN', 'CD'),
 ('CD', 'IN'),
 ('IN', 'NNS'),
 ('NNS', 'NN'),
 ('NNS', 'WDT'),
 ('WDT', 'VBP'),
 ('VBP', 'TO'),
 ('TO', 'DT'),
 ('VBN', 'RP'),
 ('RP', 'RB'),
 ('TO', 'PRP$'),
 ('PRP$', 'NN'),
 ('WP$', '

In [43]:
with open('bigram_list.json', 'w') as f:
    json.dump(bigram_list_updated, f)

In [30]:
dummy = bigram_list[0]['body'].copy()
new_dict = {}

In [32]:
for key in dummy.keys():
    new_dict[str(key)] = dummy[key]

In [33]:
dummy

Counter({('NN', 'POS'): 52,
         ('POS', 'DT'): 1,
         ('DT', 'NN'): 3649,
         ('NN', 'IN'): 2026,
         ('IN', 'DT'): 3376,
         ('IN', 'JJ'): 418,
         ('JJ', 'NN'): 1965,
         ('NN', 'DT'): 42,
         ('NN', 'VBZ'): 155,
         ('VBZ', 'IN'): 50,
         ('IN', 'NN'): 1190,
         ('NN', 'RB'): 143,
         ('RB', 'IN'): 457,
         ('NN', 'CC'): 254,
         ('CC', 'IN'): 100,
         ('IN', 'RB'): 154,
         ('RB', 'DT'): 159,
         ('DT', 'NNS'): 565,
         ('NNS', 'RB'): 51,
         ('PRP', 'MD'): 128,
         ('MD', 'VB'): 403,
         ('VB', 'PRP'): 140,
         ('PRP', 'RB'): 63,
         ('RB', 'CC'): 46,
         ('CC', 'VB'): 138,
         ('PRP', 'IN'): 156,
         ('NNS', 'IN'): 384,
         ('NN', 'VBD'): 1864,
         ('VBD', 'IN'): 533,
         ('CC', 'NN'): 317,
         ('NN', 'VBP'): 115,
         ('VBP', 'NN'): 27,
         ('JJ', 'CD'): 19,
         ('CD', 'NNP'): 4,
         ('NNP', 'VBD'): 3,
         (

In [34]:
new_dict

{"('NN', 'NN')": 497,
 "('NN', 'POS')": 52,
 "('POS', 'DT')": 1,
 "('DT', 'NN')": 3649,
 "('NN', 'IN')": 2026,
 "('IN', 'DT')": 3376,
 "('IN', 'JJ')": 418,
 "('JJ', 'NN')": 1965,
 "('NN', 'DT')": 42,
 "('NN', 'VBZ')": 155,
 "('VBZ', 'IN')": 50,
 "('IN', 'NN')": 1190,
 "('NN', 'RB')": 143,
 "('RB', 'IN')": 457,
 "('NN', 'CC')": 254,
 "('CC', 'IN')": 100,
 "('IN', 'RB')": 154,
 "('RB', 'DT')": 159,
 "('DT', 'NNS')": 565,
 "('NNS', 'RB')": 51,
 "('PRP', 'MD')": 128,
 "('MD', 'VB')": 403,
 "('VB', 'PRP')": 140,
 "('PRP', 'RB')": 63,
 "('RB', 'CC')": 46,
 "('CC', 'VB')": 138,
 "('PRP', 'IN')": 156,
 "('NNS', 'IN')": 384,
 "('NN', 'VBD')": 1864,
 "('VBD', 'IN')": 533,
 "('CC', 'NN')": 317,
 "('NN', 'VBP')": 115,
 "('VBP', 'NN')": 27,
 "('JJ', 'CD')": 19,
 "('CD', 'NNP')": 4,
 "('NNP', 'VBD')": 3,
 "('CD', 'NN')": 76,
 "('NN', 'JJ')": 119,
 "('JJ', 'JJ')": 151,
 "('NN', 'VBN')": 84,
 "('VBN', 'NN')": 69,
 "('NN', 'NNP')": 9,
 "('NNP', 'NNP')": 17,
 "('NNP', 'NN')": 10,
 "('VBZ', 'VB')": 3,
 "

In [46]:
df = pd.DataFrame(bigram_list_updated) 

In [47]:
df.head()

index                                              title  \
0  10002                        the house on the borderland   
1  10005  a voyage to the moon\r\nwith some account of t...   
2  10008                                        the mystery   
3  10046                              salute to adventurers   
4  10067  the mystery of the boule cabinet: a detective ...   

                     author  \
0     hodgson, william hope   
1            tucker, george   
2     adams, samuel hopkins   
3              buchan, john   
4  stevenson, burton egbert   

                                            subjects  \
0                                  [Science fiction]   
1  [Science fiction, Space flight to the moon -- ...   
2                                  [Science fiction]   
3  [Historical fiction, Virginia -- History -- Co...   
4           [Fiction, Detective and mystery stories]   

                                                body  
0  {'('NN', 'NN')': 497, '('NN', 'POS')': 52, '('...  
1  {'('DT', 'NN')': 4636, '('NN', 'JJ')': 113, '(...  
2  {'('DT', 'NN')': 4932, '('NN', 'JJ')': 162, '(...  
3  {'('DT', 'NN')': 6544, '('NN', 'JJ')': 153, '(...  
4  {'('DT', 'NN')': 4522, '('NN', 'JJ')': 150, '(...

In [48]:
df_try = pd.DataFrame()

In [49]:
len(bigram_list_updated[0]['body'])

539

In [55]:
len(bigram_list_updated[300]['body'])

651

In [56]:
bigram_combi = []
for entry in bigram_list_updated:
    for key in entry['body'].keys():
        bigram_combi.append(key)

In [57]:
bigram_combi = list(set(bigram_combi))

In [58]:
len(bigram_combi)

1118

In [59]:
bigram_combi

["('JJS', 'TO')",
 "('FW', 'MD')",
 "('EX', 'FW')",
 "('DT', 'JJR')",
 "('EX', 'VBG')",
 "('UH', 'PDT')",
 "('JJS', 'NN')",
 "('PDT', 'RP')",
 "('DT', 'RP')",
 "('NNP', 'VBN')",
 "('IN', 'WDT')",
 "('CC', 'MD')",
 "('DT', 'PDT')",
 "('VBZ', 'NNP')",
 "('PRP', 'VBP')",
 "('NNPS', 'VBZ')",
 "('IN', 'RB')",
 "('PRP$', 'NNS')",
 "('VB', 'WDT')",
 "('JJ', 'RP')",
 "('RB', 'CC')",
 "('WP', 'DT')",
 "('WDT', 'VBG')",
 "('VB', 'JJR')",
 "('RB', 'RBS')",
 "('NNPS', 'VBP')",
 "('POS', 'PDT')",
 "('WRB', 'RBR')",
 "('TO', 'FW')",
 "('EX', 'WP$')",
 "('NNP', 'RP')",
 "('VB', 'IN')",
 "('FW', 'NNS')",
 "('VBD', 'VB')",
 "('CC', 'JJR')",
 "('VBN', 'DT')",
 "('DT', 'VBN')",
 "('PRP$', 'TO')",
 "('TO', 'EX')",
 "('VBN', 'RP')",
 "('WRB', 'WDT')",
 "('VBP', 'UH')",
 "('JJ', 'NNS')",
 "('RB', 'VBG')",
 "('VB', 'JJS')",
 "('TO', 'NNS')",
 "('RBR', 'EX')",
 "('EX', 'PDT')",
 "('CC', 'PRP')",
 "('RBS', 'JJ')",
 "('CD', 'NNP')",
 "('CD', 'WP')",
 "('RBR', 'POS')",
 "('RP', 'VB')",
 "('DT', 'NNPS')",
 "('MD'

In [60]:
df_cols = bigram_combi + ['subjects']

In [61]:
df = pd.DataFrame(columns=df_cols)

In [62]:
subjects = {'Adventure stories': 1, 'Detective and mystery stories': 2, 'Historical fiction': 3, 'Love stories': 4,
           'Science fiction': 5, 'Western stories': 6}

In [64]:
row = 0
for entry in bigram_list_updated:
    for bigram in bigram_combi:
        if bigram in entry['body'].keys():
            df.loc[row, bigram] = entry['body'][bigram]
        else:
            df.loc[row, bigram] = 0
    for subject in entry['subjects']:
        if subject in subjects:
            df.loc[row, 'subjects'] = subjects[subject]
            break
    row = row + 1

In [65]:
df.head()

('JJS', 'TO') ('FW', 'MD') ('EX', 'FW') ('DT', 'JJR') ('EX', 'VBG')  \
0             1            0            0            28             0   
1             2            0            0            60             1   
2             4            0            0            32             1   
3             8            0            0            74             0   
4             0            2            0            29             7   

  ('UH', 'PDT') ('JJS', 'NN') ('PDT', 'RP') ('DT', 'RP') ('NNP', 'VBN')  ...  \
0             0            10             0            0              1  ...   
1             0            49             0            0              1  ...   
2             0            33             0            2              1  ...   
3             0            76             0            0              1  ...   
4             0            46             0            5              1  ...   

  ('UH', 'RB') ('PRP', 'RBR') ('NN', 'NNPS') ('VBG', 'PRP') ('NN', 'CD')  \
0            0              4              0             59           23   
1            0              5              0            102           28   
2            0              5              0             64           40   
3            0              3              0             66           34   
4            0              6              0             92           28   

  ('VBN', 'CD') ('IN', 'WP') ('RBS', 'VBN') ('RBR', 'TO') subjects  
0             5           14              0             2        5  
1             9           64              1             2        5  
2             7           27              0             2        5  
3             4           46              0             4        3  
4            10           35              1             2        2  

[5 rows x 1119 columns]

In [69]:
df.shape

(3889, 1119)

In [70]:
index = []
for entry in bigram_list_updated:
    index.append(entry['index'])

In [71]:
index

['10002',
 '10005',
 '10008',
 '10046',
 '10067',
 '10094',
 '10109',
 '10124',
 '1014',
 '10234',
 '10324',
 '10371',
 '10372',
 '10391',
 '10402',
 '10418',
 '10434',
 '10443',
 '10452',
 '10455',
 '10465',
 '10473',
 '10476',
 '105',
 '10509',
 '10542',
 '10584',
 '10585',
 '106',
 '10601',
 '10608',
 '10662',
 '10743',
 '1083',
 '10882',
 '10886',
 '10929',
 '10988',
 '11043',
 '11082',
 '11085',
 '11106',
 '11110',
 '11127',
 '11128',
 '11166',
 '11237',
 '11239',
 '11373',
 '11418',
 '11451',
 '1153',
 '11556',
 '1159',
 '11595',
 '11609',
 '11626',
 '11696',
 '11752',
 '11860',
 '11869',
 '11901',
 '11909',
 '1194',
 '11957',
 '11997',
 '1206',
 '1214',
 '12144',
 '12175',
 '12187',
 '12189',
 '12190',
 '12192',
 '12239',
 '12256',
 '12259',
 '12265',
 '12281',
 '12349',
 '12388',
 '1239',
 '12405',
 '12436',
 '12442',
 '12484',
 '12495',
 '12509',
 '12535',
 '12556',
 '12557',
 '12558',
 '12560',
 '12587',
 '126',
 '1261',
 '12611',
 '1262',
 '12623',
 '1263',
 '12663',
 '12678

In [73]:
df.index = index
df.head()

('JJS', 'TO') ('FW', 'MD') ('EX', 'FW') ('DT', 'JJR') ('EX', 'VBG')  \
10002             1            0            0            28             0   
10005             2            0            0            60             1   
10008             4            0            0            32             1   
10046             8            0            0            74             0   
10067             0            2            0            29             7   

      ('UH', 'PDT') ('JJS', 'NN') ('PDT', 'RP') ('DT', 'RP') ('NNP', 'VBN')  \
10002             0            10             0            0              1   
10005             0            49             0            0              1   
10008             0            33             0            2              1   
10046             0            76             0            0              1   
10067             0            46             0            5              1   

       ... ('UH', 'RB') ('PRP', 'RBR') ('NN', 'NNPS') ('VBG', 'PRP')  \
10002  ...            0              4              0             59   
10005  ...            0              5              0            102   
10008  ...            0              5              0             64   
10046  ...            0              3              0             66   
10067  ...            0              6              0             92   

      ('NN', 'CD') ('VBN', 'CD') ('IN', 'WP') ('RBS', 'VBN') ('RBR', 'TO')  \
10002           23             5           14              0             2   
10005           28             9           64              1             2   
10008           40             7           27              0             2   
10046           34             4           46              0             4   
10067           28            10           35              1             2   

      subjects  
10002        5  
10005        5  
10008        5  
10046        3  
10067        2  

[5 rows x 1119 columns]

In [74]:
df.to_csv('bigram.csv')